# Initialisation of variables and libraries


In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

In [2]:
from torch import nn
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms

device = "cuda:0" if torch.cuda.is_available() else "cpu"
# Hyper-parameters
T = 10
epsilon = 16
m = 5
mu = 1.0
kernel = (torch.Tensor([[0, 0, 1, 2, 1, 0, 0], [0, 3, 13, 22, 13, 3, 0], [1, 13, 59, 97, 59, 13, 1], [2, 22, 97, 159, 97, 22, 2], [1, 13, 59, 97, 59, 13, 1], [0, 3, 13, 22, 13, 3, 0], [0, 0, 1, 2, 1, 0, 0]]) / 1003).to(device)
trans_prob = 0.5
batch_size = 8

In [ ]:
# Reading the database from the google drive
from google.colab import drive
drive.mount('/content/gdrive')
!unzip gdrive/MyDrive/DLDataset.zip
print()

In [4]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from PIL import Image

# Custom Dataset, which works with how the data is saved
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])

        image = Image.open(img_path)

        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        
        if self.target_transform:
            label = self.target_transform(label)
        
        label-=1

        return image, label

In [5]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms 

# Creation of the data loader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

annotations_file = "dev_data/val_rs.csv"
img_dir = "dev_data/val_rs/"

train_dataset = CustomImageDataset(annotations_file, img_dir, transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

# Models

###Inception V3

In [6]:
inc_v3 = torch.hub.load('pytorch/vision', 'inception_v3', pretrained=True).to(device)
inc_v3.eval()
print()

Downloading: "https://github.com/pytorch/vision/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


### Inception V4
https://github.com/Cadene/pretrained-models.pytorch/blob/master/pretrainedmodels/models/inceptionv4.py

In [7]:
from __future__ import print_function, division, absolute_import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
import os
import sys

__all__ = ['InceptionV4', 'inceptionv4']

pretrained_settings = {
    'inceptionv4': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/inceptionv4-8e4777a0.pth',
            'input_space': 'RGB',
            'input_size': [3, 299, 299],
            'input_range': [0, 1],
            'mean': [0.5, 0.5, 0.5],
            'std': [0.5, 0.5, 0.5],
            'num_classes': 1000
        },
        'imagenet+background': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/inceptionv4-8e4777a0.pth',
            'input_space': 'RGB',
            'input_size': [3, 299, 299],
            'input_range': [0, 1],
            'mean': [0.5, 0.5, 0.5],
            'std': [0.5, 0.5, 0.5],
            'num_classes': 1001
        }
    }
}


class BasicConv2d(nn.Module):

    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes,
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, bias=False) # verify bias false
        self.bn = nn.BatchNorm2d(out_planes,
                                 eps=0.001, # value found in tensorflow
                                 momentum=0.1, # default pytorch value
                                 affine=True)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


class Mixed_3a(nn.Module):

    def __init__(self):
        super(Mixed_3a, self).__init__()
        self.maxpool = nn.MaxPool2d(3, stride=2)
        self.conv = BasicConv2d(64, 96, kernel_size=3, stride=2)

    def forward(self, x):
        x0 = self.maxpool(x)
        x1 = self.conv(x)
        out = torch.cat((x0, x1), 1)
        return out


class Mixed_4a(nn.Module):

    def __init__(self):
        super(Mixed_4a, self).__init__()

        self.branch0 = nn.Sequential(
            BasicConv2d(160, 64, kernel_size=1, stride=1),
            BasicConv2d(64, 96, kernel_size=3, stride=1)
        )

        self.branch1 = nn.Sequential(
            BasicConv2d(160, 64, kernel_size=1, stride=1),
            BasicConv2d(64, 64, kernel_size=(1,7), stride=1, padding=(0,3)),
            BasicConv2d(64, 64, kernel_size=(7,1), stride=1, padding=(3,0)),
            BasicConv2d(64, 96, kernel_size=(3,3), stride=1)
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        out = torch.cat((x0, x1), 1)
        return out


class Mixed_5a(nn.Module):

    def __init__(self):
        super(Mixed_5a, self).__init__()
        self.conv = BasicConv2d(192, 192, kernel_size=3, stride=2)
        self.maxpool = nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        x0 = self.conv(x)
        x1 = self.maxpool(x)
        out = torch.cat((x0, x1), 1)
        return out


class Inception_A(nn.Module):

    def __init__(self):
        super(Inception_A, self).__init__()
        self.branch0 = BasicConv2d(384, 96, kernel_size=1, stride=1)

        self.branch1 = nn.Sequential(
            BasicConv2d(384, 64, kernel_size=1, stride=1),
            BasicConv2d(64, 96, kernel_size=3, stride=1, padding=1)
        )

        self.branch2 = nn.Sequential(
            BasicConv2d(384, 64, kernel_size=1, stride=1),
            BasicConv2d(64, 96, kernel_size=3, stride=1, padding=1),
            BasicConv2d(96, 96, kernel_size=3, stride=1, padding=1)
        )

        self.branch3 = nn.Sequential(
            nn.AvgPool2d(3, stride=1, padding=1, count_include_pad=False),
            BasicConv2d(384, 96, kernel_size=1, stride=1)
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class Reduction_A(nn.Module):

    def __init__(self):
        super(Reduction_A, self).__init__()
        self.branch0 = BasicConv2d(384, 384, kernel_size=3, stride=2)

        self.branch1 = nn.Sequential(
            BasicConv2d(384, 192, kernel_size=1, stride=1),
            BasicConv2d(192, 224, kernel_size=3, stride=1, padding=1),
            BasicConv2d(224, 256, kernel_size=3, stride=2)
        )

        self.branch2 = nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        out = torch.cat((x0, x1, x2), 1)
        return out


class Inception_B(nn.Module):

    def __init__(self):
        super(Inception_B, self).__init__()
        self.branch0 = BasicConv2d(1024, 384, kernel_size=1, stride=1)

        self.branch1 = nn.Sequential(
            BasicConv2d(1024, 192, kernel_size=1, stride=1),
            BasicConv2d(192, 224, kernel_size=(1,7), stride=1, padding=(0,3)),
            BasicConv2d(224, 256, kernel_size=(7,1), stride=1, padding=(3,0))
        )

        self.branch2 = nn.Sequential(
            BasicConv2d(1024, 192, kernel_size=1, stride=1),
            BasicConv2d(192, 192, kernel_size=(7,1), stride=1, padding=(3,0)),
            BasicConv2d(192, 224, kernel_size=(1,7), stride=1, padding=(0,3)),
            BasicConv2d(224, 224, kernel_size=(7,1), stride=1, padding=(3,0)),
            BasicConv2d(224, 256, kernel_size=(1,7), stride=1, padding=(0,3))
        )

        self.branch3 = nn.Sequential(
            nn.AvgPool2d(3, stride=1, padding=1, count_include_pad=False),
            BasicConv2d(1024, 128, kernel_size=1, stride=1)
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class Reduction_B(nn.Module):

    def __init__(self):
        super(Reduction_B, self).__init__()

        self.branch0 = nn.Sequential(
            BasicConv2d(1024, 192, kernel_size=1, stride=1),
            BasicConv2d(192, 192, kernel_size=3, stride=2)
        )

        self.branch1 = nn.Sequential(
            BasicConv2d(1024, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 256, kernel_size=(1,7), stride=1, padding=(0,3)),
            BasicConv2d(256, 320, kernel_size=(7,1), stride=1, padding=(3,0)),
            BasicConv2d(320, 320, kernel_size=3, stride=2)
        )

        self.branch2 = nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        out = torch.cat((x0, x1, x2), 1)
        return out


class Inception_C(nn.Module):

    def __init__(self):
        super(Inception_C, self).__init__()

        self.branch0 = BasicConv2d(1536, 256, kernel_size=1, stride=1)

        self.branch1_0 = BasicConv2d(1536, 384, kernel_size=1, stride=1)
        self.branch1_1a = BasicConv2d(384, 256, kernel_size=(1,3), stride=1, padding=(0,1))
        self.branch1_1b = BasicConv2d(384, 256, kernel_size=(3,1), stride=1, padding=(1,0))

        self.branch2_0 = BasicConv2d(1536, 384, kernel_size=1, stride=1)
        self.branch2_1 = BasicConv2d(384, 448, kernel_size=(3,1), stride=1, padding=(1,0))
        self.branch2_2 = BasicConv2d(448, 512, kernel_size=(1,3), stride=1, padding=(0,1))
        self.branch2_3a = BasicConv2d(512, 256, kernel_size=(1,3), stride=1, padding=(0,1))
        self.branch2_3b = BasicConv2d(512, 256, kernel_size=(3,1), stride=1, padding=(1,0))

        self.branch3 = nn.Sequential(
            nn.AvgPool2d(3, stride=1, padding=1, count_include_pad=False),
            BasicConv2d(1536, 256, kernel_size=1, stride=1)
        )

    def forward(self, x):
        x0 = self.branch0(x)

        x1_0 = self.branch1_0(x)
        x1_1a = self.branch1_1a(x1_0)
        x1_1b = self.branch1_1b(x1_0)
        x1 = torch.cat((x1_1a, x1_1b), 1)

        x2_0 = self.branch2_0(x)
        x2_1 = self.branch2_1(x2_0)
        x2_2 = self.branch2_2(x2_1)
        x2_3a = self.branch2_3a(x2_2)
        x2_3b = self.branch2_3b(x2_2)
        x2 = torch.cat((x2_3a, x2_3b), 1)

        x3 = self.branch3(x)

        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class InceptionV4(nn.Module):

    def __init__(self, num_classes=1001):
        super(InceptionV4, self).__init__()
        # Special attributs
        self.input_space = None
        self.input_size = (299, 299, 3)
        self.mean = None
        self.std = None
        # Modules
        self.features = nn.Sequential(
            BasicConv2d(3, 32, kernel_size=3, stride=2),
            BasicConv2d(32, 32, kernel_size=3, stride=1),
            BasicConv2d(32, 64, kernel_size=3, stride=1, padding=1),
            Mixed_3a(),
            Mixed_4a(),
            Mixed_5a(),
            Inception_A(),
            Inception_A(),
            Inception_A(),
            Inception_A(),
            Reduction_A(), # Mixed_6a
            Inception_B(),
            Inception_B(),
            Inception_B(),
            Inception_B(),
            Inception_B(),
            Inception_B(),
            Inception_B(),
            Reduction_B(), # Mixed_7a
            Inception_C(),
            Inception_C(),
            Inception_C()
        )
        self.last_linear = nn.Linear(1536, num_classes)

    def logits(self, features):
        #Allows image of any size to be processed
        adaptiveAvgPoolWidth = features.shape[2]
        x = F.avg_pool2d(features, kernel_size=adaptiveAvgPoolWidth)
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, input):
        x = self.features(input)
        x = self.logits(x)
        return x


def inceptionv4(num_classes=1000, pretrained='imagenet'):
    if pretrained:
        settings = pretrained_settings['inceptionv4'][pretrained]
        assert num_classes == settings['num_classes'], \
            "num_classes should be {}, but is {}".format(settings['num_classes'], num_classes)

        # both 'imagenet'&'imagenet+background' are loaded from same parameters
        model = InceptionV4(num_classes=1001)
        model.load_state_dict(model_zoo.load_url(settings['url']))

        if pretrained == 'imagenet':
            new_last_linear = nn.Linear(1536, 1000)
            new_last_linear.weight.data = model.last_linear.weight.data[1:]
            new_last_linear.bias.data = model.last_linear.bias.data[1:]
            model.last_linear = new_last_linear

        model.input_space = settings['input_space']
        model.input_size = settings['input_size']
        model.input_range = settings['input_range']
        model.mean = settings['mean']
        model.std = settings['std']
    else:
        model = InceptionV4(num_classes=num_classes)
    return model

In [8]:
inc_v4 = inceptionv4(num_classes=1000, pretrained='imagenet').to(device)
inc_v4.eval()
print()

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/inceptionv4-8e4777a0.pth" to /root/.cache/torch/hub/checkpoints/inceptionv4-8e4777a0.pth


### Inception-Resnet-v2
https://github.com/Cadene/pretrained-models.pytorch/blob/master/pretrainedmodels/models/inceptionresnetv2.py

In [9]:
from __future__ import print_function, division, absolute_import
import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import os
import sys

__all__ = ['InceptionResNetV2', 'inceptionresnetv2']

pretrained_settings = {
    'inceptionresnetv2': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/inceptionresnetv2-520b38e4.pth',
            'input_space': 'RGB',
            'input_size': [3, 299, 299],
            'input_range': [0, 1],
            'mean': [0.5, 0.5, 0.5],
            'std': [0.5, 0.5, 0.5],
            'num_classes': 1000
        },
        'imagenet+background': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/inceptionresnetv2-520b38e4.pth',
            'input_space': 'RGB',
            'input_size': [3, 299, 299],
            'input_range': [0, 1],
            'mean': [0.5, 0.5, 0.5],
            'std': [0.5, 0.5, 0.5],
            'num_classes': 1001
        }
    }
}


class BasicConv2d(nn.Module):

    def __init__(self, in_planes, out_planes, kernel_size, stride, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes,
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, bias=False) # verify bias false
        self.bn = nn.BatchNorm2d(out_planes,
                                 eps=0.001, # value found in tensorflow
                                 momentum=0.1, # default pytorch value
                                 affine=True)
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x


class Mixed_5b(nn.Module):

    def __init__(self):
        super(Mixed_5b, self).__init__()

        self.branch0 = BasicConv2d(192, 96, kernel_size=1, stride=1)

        self.branch1 = nn.Sequential(
            BasicConv2d(192, 48, kernel_size=1, stride=1),
            BasicConv2d(48, 64, kernel_size=5, stride=1, padding=2)
        )

        self.branch2 = nn.Sequential(
            BasicConv2d(192, 64, kernel_size=1, stride=1),
            BasicConv2d(64, 96, kernel_size=3, stride=1, padding=1),
            BasicConv2d(96, 96, kernel_size=3, stride=1, padding=1)
        )

        self.branch3 = nn.Sequential(
            nn.AvgPool2d(3, stride=1, padding=1, count_include_pad=False),
            BasicConv2d(192, 64, kernel_size=1, stride=1)
        )

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class Block35(nn.Module):

    def __init__(self, scale=1.0):
        super(Block35, self).__init__()

        self.scale = scale

        self.branch0 = BasicConv2d(320, 32, kernel_size=1, stride=1)

        self.branch1 = nn.Sequential(
            BasicConv2d(320, 32, kernel_size=1, stride=1),
            BasicConv2d(32, 32, kernel_size=3, stride=1, padding=1)
        )

        self.branch2 = nn.Sequential(
            BasicConv2d(320, 32, kernel_size=1, stride=1),
            BasicConv2d(32, 48, kernel_size=3, stride=1, padding=1),
            BasicConv2d(48, 64, kernel_size=3, stride=1, padding=1)
        )

        self.conv2d = nn.Conv2d(128, 320, kernel_size=1, stride=1)
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        out = torch.cat((x0, x1, x2), 1)
        out = self.conv2d(out)
        out = out * self.scale + x
        out = self.relu(out)
        return out


class Mixed_6a(nn.Module):

    def __init__(self):
        super(Mixed_6a, self).__init__()

        self.branch0 = BasicConv2d(320, 384, kernel_size=3, stride=2)

        self.branch1 = nn.Sequential(
            BasicConv2d(320, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 256, kernel_size=3, stride=1, padding=1),
            BasicConv2d(256, 384, kernel_size=3, stride=2)
        )

        self.branch2 = nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        out = torch.cat((x0, x1, x2), 1)
        return out


class Block17(nn.Module):

    def __init__(self, scale=1.0):
        super(Block17, self).__init__()

        self.scale = scale

        self.branch0 = BasicConv2d(1088, 192, kernel_size=1, stride=1)

        self.branch1 = nn.Sequential(
            BasicConv2d(1088, 128, kernel_size=1, stride=1),
            BasicConv2d(128, 160, kernel_size=(1,7), stride=1, padding=(0,3)),
            BasicConv2d(160, 192, kernel_size=(7,1), stride=1, padding=(3,0))
        )

        self.conv2d = nn.Conv2d(384, 1088, kernel_size=1, stride=1)
        self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        out = torch.cat((x0, x1), 1)
        out = self.conv2d(out)
        out = out * self.scale + x
        out = self.relu(out)
        return out


class Mixed_7a(nn.Module):

    def __init__(self):
        super(Mixed_7a, self).__init__()

        self.branch0 = nn.Sequential(
            BasicConv2d(1088, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 384, kernel_size=3, stride=2)
        )

        self.branch1 = nn.Sequential(
            BasicConv2d(1088, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 288, kernel_size=3, stride=2)
        )

        self.branch2 = nn.Sequential(
            BasicConv2d(1088, 256, kernel_size=1, stride=1),
            BasicConv2d(256, 288, kernel_size=3, stride=1, padding=1),
            BasicConv2d(288, 320, kernel_size=3, stride=2)
        )

        self.branch3 = nn.MaxPool2d(3, stride=2)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out


class Block8(nn.Module):

    def __init__(self, scale=1.0, noReLU=False):
        super(Block8, self).__init__()

        self.scale = scale
        self.noReLU = noReLU

        self.branch0 = BasicConv2d(2080, 192, kernel_size=1, stride=1)

        self.branch1 = nn.Sequential(
            BasicConv2d(2080, 192, kernel_size=1, stride=1),
            BasicConv2d(192, 224, kernel_size=(1,3), stride=1, padding=(0,1)),
            BasicConv2d(224, 256, kernel_size=(3,1), stride=1, padding=(1,0))
        )

        self.conv2d = nn.Conv2d(448, 2080, kernel_size=1, stride=1)
        if not self.noReLU:
            self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x0 = self.branch0(x)
        x1 = self.branch1(x)
        out = torch.cat((x0, x1), 1)
        out = self.conv2d(out)
        out = out * self.scale + x
        if not self.noReLU:
            out = self.relu(out)
        return out


class InceptionResNetV2(nn.Module):

    def __init__(self, num_classes=1001):
        super(InceptionResNetV2, self).__init__()
        # Special attributs
        self.input_space = None
        self.input_size = (299, 299, 3)
        self.mean = None
        self.std = None
        # Modules
        self.conv2d_1a = BasicConv2d(3, 32, kernel_size=3, stride=2)
        self.conv2d_2a = BasicConv2d(32, 32, kernel_size=3, stride=1)
        self.conv2d_2b = BasicConv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.maxpool_3a = nn.MaxPool2d(3, stride=2)
        self.conv2d_3b = BasicConv2d(64, 80, kernel_size=1, stride=1)
        self.conv2d_4a = BasicConv2d(80, 192, kernel_size=3, stride=1)
        self.maxpool_5a = nn.MaxPool2d(3, stride=2)
        self.mixed_5b = Mixed_5b()
        self.repeat = nn.Sequential(
            Block35(scale=0.17),
            Block35(scale=0.17),
            Block35(scale=0.17),
            Block35(scale=0.17),
            Block35(scale=0.17),
            Block35(scale=0.17),
            Block35(scale=0.17),
            Block35(scale=0.17),
            Block35(scale=0.17),
            Block35(scale=0.17)
        )
        self.mixed_6a = Mixed_6a()
        self.repeat_1 = nn.Sequential(
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10),
            Block17(scale=0.10)
        )
        self.mixed_7a = Mixed_7a()
        self.repeat_2 = nn.Sequential(
            Block8(scale=0.20),
            Block8(scale=0.20),
            Block8(scale=0.20),
            Block8(scale=0.20),
            Block8(scale=0.20),
            Block8(scale=0.20),
            Block8(scale=0.20),
            Block8(scale=0.20),
            Block8(scale=0.20)
        )
        self.block8 = Block8(noReLU=True)
        self.conv2d_7b = BasicConv2d(2080, 1536, kernel_size=1, stride=1)
        self.avgpool_1a = nn.AvgPool2d(8, count_include_pad=False)
        self.last_linear = nn.Linear(1536, num_classes)

    def features(self, input):
        x = self.conv2d_1a(input)
        x = self.conv2d_2a(x)
        x = self.conv2d_2b(x)
        x = self.maxpool_3a(x)
        x = self.conv2d_3b(x)
        x = self.conv2d_4a(x)
        x = self.maxpool_5a(x)
        x = self.mixed_5b(x)
        x = self.repeat(x)
        x = self.mixed_6a(x)
        x = self.repeat_1(x)
        x = self.mixed_7a(x)
        x = self.repeat_2(x)
        x = self.block8(x)
        x = self.conv2d_7b(x)
        return x

    def logits(self, features):
        x = self.avgpool_1a(features)
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, input):
        x = self.features(input)
        x = self.logits(x)
        return x

def inceptionresnetv2(num_classes=1000, pretrained='imagenet'):
    r"""InceptionResNetV2 model architecture from the
    `"InceptionV4, Inception-ResNet..." <https://arxiv.org/abs/1602.07261>`_ paper.
    """
    if pretrained:
        settings = pretrained_settings['inceptionresnetv2'][pretrained]
        assert num_classes == settings['num_classes'], \
            "num_classes should be {}, but is {}".format(settings['num_classes'], num_classes)

        # both 'imagenet'&'imagenet+background' are loaded from same parameters
        model = InceptionResNetV2(num_classes=1001)
        model.load_state_dict(model_zoo.load_url(settings['url']))

        if pretrained == 'imagenet':
            new_last_linear = nn.Linear(1536, 1000)
            new_last_linear.weight.data = model.last_linear.weight.data[1:]
            new_last_linear.bias.data = model.last_linear.bias.data[1:]
            model.last_linear = new_last_linear

        model.input_space = settings['input_space']
        model.input_size = settings['input_size']
        model.input_range = settings['input_range']

        model.mean = settings['mean']
        model.std = settings['std']
    else:
        model = InceptionResNetV2(num_classes=num_classes)
    return model

In [10]:
inc_Res_v2 = inceptionresnetv2(num_classes=1000, pretrained='imagenet').to(device)
inc_Res_v2.eval()
print()

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/inceptionresnetv2-520b38e4.pth" to /root/.cache/torch/hub/checkpoints/inceptionresnetv2-520b38e4.pth


### Resnet-v2-101

In [11]:
import torchvision.models as models
resnet101 = models.resnet101(pretrained=True).to(device)
resnet101.eval()
print()

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth



### Inception V3 Adversarially Trained

In [12]:
try:
    import timm
except:
    !pip install timm
    import timm

inc_v3_adv = timm.create_model('adv_inception_v3', pretrained=True).to(device)
inc_v3_adv.eval()
print()

     |████████████████████████████████| 296kB 8.8MB/s 


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/adv_inception_v3-9e27bd63.pth" to /root/.cache/torch/hub/checkpoints/adv_inception_v3-9e27bd63.pth


# Reimplementations

## Table 1 reimplementation

### (a)SI-NI-TIM

In [ ]:
def SI_NI_TIM_1(x, y, model):
    pred = model(x)
    y_pred = torch.argmax(pred, dim=1)
    eps = epsilon/255
    alpha = eps/T
    g = torch.zeros_like(x).to(device)
    x_adv = torch.clone(x)
    crit = nn.CrossEntropyLoss()
    for i in range(T):
        grads = torch.zeros_like(x).to(device)
        x_nes = x_adv + alpha*mu*g
        for j in range(m):
            x_s = (x_nes / (j+1)).requires_grad_(True)
            x_s.grad = None
            pred = model(x_s)
            loss = crit(pred, y)
            loss.backward()
            grads += x_s.grad
        grads /= m
        gr = torch.zeros_like(grads).to(device)
        grads = F.pad(grads, (3, 4, 3, 4), 'constant', 0)
        for j in range(7):
            for k in range(7):
                gr += grads[:, :, j:-7+j, k:-7+k]*kernel[j, k]
        grads = gr/torch.mean(torch.abs(gr), (1, 2, 3), keepdim=True)
        g = mu*g + grads
        x_adv += alpha * torch.sign(g)
        dif = x - x_adv
        dif = torch.clamp(dif, -eps, eps)
        x_adv = x - dif
    pred = model(x_adv)
    y_guess = torch.argmax(pred, dim=1)
    return x_adv, torch.count_nonzero(y_guess-y_pred)

In [ ]:
# white box Inc v3
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_TIM_1(x.to(device), y.to(device), inc_v3)
    ta[0] += acc

    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_v4(x_advs)
    y_incv4_after = torch.argmax(pred, dim=1)
    ta[1] += torch.count_nonzero(y_incv4_after-y_incv4)

    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_advs)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    ta[2] += torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2)

    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    pred = resnet101(x_advs)
    y_resnet101_after = torch.argmax(pred, dim=1)
    ta[3] += torch.count_nonzero(y_resnet101_after-y_resnet101)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)

In [ ]:
# white box Inc v4
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_TIM_1(x.to(device), y.to(device), inc_v4)
    ta[1] += acc

    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[0] += torch.count_nonzero(y_incv3_after-y_incv3)

    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_advs)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    ta[2] += torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2)

    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    pred = resnet101(x_advs)
    y_resnet101_after = torch.argmax(pred, dim=1)
    ta[3] += torch.count_nonzero(y_resnet101_after-y_resnet101)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)

In [ ]:
# white box Inc Res v2
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_TIM_1(x.to(device), y.to(device), inc_Res_v2)
    ta[2] += acc

    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[0] += torch.count_nonzero(y_incv3_after-y_incv3)

    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_v4(x_advs)
    y_incv4_after = torch.argmax(pred, dim=1)
    ta[1] += torch.count_nonzero(y_incv4_after-y_incv4)

    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    pred = resnet101(x_advs)
    y_resnet101_after = torch.argmax(pred, dim=1)
    ta[3] += torch.count_nonzero(y_resnet101_after-y_resnet101)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)

In [ ]:
# white box Resnet101
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_TIM_1(x.to(device), y.to(device), resnet101)
    ta[3] += acc

    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[0] += torch.count_nonzero(y_incv3_after-y_incv3)

    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_v4(x_advs)
    y_incv4_after = torch.argmax(pred, dim=1)
    ta[1] += torch.count_nonzero(y_incv4_after-y_incv4)

    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_advs)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    ta[2] += torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)

### (b)SI-NI-DIM

In [ ]:
def trans(x):
    size = torch.randint(299, 331, ())
    x = F.interpolate(x, size=(size, size))
    dif = 331 - size
    left = torch.randint(0, dif, ())
    right = dif-left
    top = torch.randint(0, dif, ())
    bottom = dif - top
    x = F.pad(x, (left, right, top, bottom), 'constant', 0)
    x = F.interpolate(x, size=(299, 299))
    return x

In [ ]:
def SI_NI_DIM_1(x, y, model):
    pred = model(x)
    y_pred = torch.argmax(pred, dim=1)
    eps = epsilon/255
    alpha = eps/T
    g = torch.zeros_like(x).to(device)
    x_adv = torch.clone(x)
    crit = nn.CrossEntropyLoss()
    for i in range(T):
        grads = torch.zeros_like(x).to(device)
        x_nes = x_adv + alpha*mu*g
        for j in range(m):
            x_s = (x_nes / (j+1))
            if torch.rand(1) > trans_prob:
                x_s = trans(x_s)
            x_s = x_s.requires_grad_(True)
            x_s.grad = None
            pred = model(x_s)
            loss = crit(pred, y)
            loss.backward()
            grads += x_s.grad
        grads /= m
        grads = grads/torch.mean(torch.abs(grads), (1, 2, 3), keepdim=True)
        g = mu*g + grads
        x_adv += alpha * torch.sign(g)
        dif = x - x_adv
        dif = torch.clamp(dif, -eps, eps)
        x_adv = x - dif
    pred = model(x_adv)
    y_guess = torch.argmax(pred, dim=1)
    return x_adv, torch.count_nonzero(y_guess-y_pred)

In [ ]:
# white box Inc v3
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_DIM_1(x.to(device), y.to(device), inc_v3)
    ta[0] += acc

    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_v4(x_advs)
    y_incv4_after = torch.argmax(pred, dim=1)
    ta[1] += torch.count_nonzero(y_incv4_after-y_incv4)

    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_advs)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    ta[2] += torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2)

    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    pred = resnet101(x_advs)
    y_resnet101_after = torch.argmax(pred, dim=1)
    ta[3] += torch.count_nonzero(y_resnet101_after-y_resnet101)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)

In [ ]:
# white box Inc v4
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_DIM_1(x.to(device), y.to(device), inc_v4)
    ta[1] += acc

    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[0] += torch.count_nonzero(y_incv3_after-y_incv3)

    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_advs)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    ta[2] += torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2)

    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    pred = resnet101(x_advs)
    y_resnet101_after = torch.argmax(pred, dim=1)
    ta[3] += torch.count_nonzero(y_resnet101_after-y_resnet101)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)

In [ ]:
# white box Inc Res v2
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_DIM_1(x.to(device), y.to(device), inc_Res_v2)
    ta[2] += acc

    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[0] += torch.count_nonzero(y_incv3_after-y_incv3)

    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_v4(x_advs)
    y_incv4_after = torch.argmax(pred, dim=1)
    ta[1] += torch.count_nonzero(y_incv4_after-y_incv4)

    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    pred = resnet101(x_advs)
    y_resnet101_after = torch.argmax(pred, dim=1)
    ta[3] += torch.count_nonzero(y_resnet101_after-y_resnet101)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)

In [ ]:
# white box Resnet101
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_DIM_1(x.to(device), y.to(device), resnet101)
    ta[3] += acc

    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[0] += torch.count_nonzero(y_incv3_after-y_incv3)

    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_v4(x_advs)
    y_incv4_after = torch.argmax(pred, dim=1)
    ta[1] += torch.count_nonzero(y_incv4_after-y_incv4)

    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_advs)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    ta[2] += torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)

### (c)SI-NI-TI-DIM

In [ ]:
def SI_NI_TI_DIM_1(x, y, model):
    pred = model(x)
    y_pred = torch.argmax(pred, dim=1)
    eps = epsilon/255
    alpha = eps/T
    g = torch.zeros_like(x).to(device)
    x_adv = torch.clone(x)
    crit = nn.CrossEntropyLoss()
    for i in range(T):
        grads = torch.zeros_like(x).to(device)
        x_nes = x_adv + alpha*mu*g
        for j in range(m):
            x_s = (x_nes / (j+1))
            if torch.rand(1) > trans_prob:
                x_s = trans(x_s)
            x_s = x_s.requires_grad_(True)
            x_s.grad = None
            pred = model(x_s)
            loss = crit(pred, y)
            loss.backward()
            grads += x_s.grad
        grads /= m
        gr = torch.zeros_like(grads).to(device)
        grads = F.pad(grads, (3, 4, 3, 4), 'constant', 0)
        for j in range(7):
            for k in range(7):
                gr += grads[:, :, j:-7+j, k:-7+k]*kernel[j, k]
        grads = gr/torch.mean(torch.abs(gr), (1, 2, 3), keepdim=True)
        g = mu*g + grads
        x_adv += alpha * torch.sign(g)
        dif = x - x_adv
        dif = torch.clamp(dif, -eps, eps)
        x_adv = x - dif
    pred = model(x_adv)
    y_guess = torch.argmax(pred, dim=1)
    return x_adv, torch.count_nonzero(y_guess-y_pred)

In [ ]:
# white box Inc v3
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_TI_DIM_1(x.to(device), y.to(device), inc_v3)
    ta[0] += acc

    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_v4(x_advs)
    y_incv4_after = torch.argmax(pred, dim=1)
    ta[1] += torch.count_nonzero(y_incv4_after-y_incv4)

    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_advs)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    ta[2] += torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2)

    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    pred = resnet101(x_advs)
    y_resnet101_after = torch.argmax(pred, dim=1)
    ta[3] += torch.count_nonzero(y_resnet101_after-y_resnet101)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)

In [ ]:
# white box Inc v4
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_TI_DIM_1(x.to(device), y.to(device), inc_v4)
    ta[1] += acc

    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[0] += torch.count_nonzero(y_incv3_after-y_incv3)

    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_advs)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    ta[2] += torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2)

    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    pred = resnet101(x_advs)
    y_resnet101_after = torch.argmax(pred, dim=1)
    ta[3] += torch.count_nonzero(y_resnet101_after-y_resnet101)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)

In [ ]:
# white box Inc Res v2
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_TI_DIM_1(x.to(device), y.to(device), inc_Res_v2)
    ta[2] += acc

    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[0] += torch.count_nonzero(y_incv3_after-y_incv3)

    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_v4(x_advs)
    y_incv4_after = torch.argmax(pred, dim=1)
    ta[1] += torch.count_nonzero(y_incv4_after-y_incv4)

    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    pred = resnet101(x_advs)
    y_resnet101_after = torch.argmax(pred, dim=1)
    ta[3] += torch.count_nonzero(y_resnet101_after-y_resnet101)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)

In [ ]:
# white box Resnet101
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_TI_DIM_1(x.to(device), y.to(device), resnet101)
    ta[3] += acc

    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[0] += torch.count_nonzero(y_incv3_after-y_incv3)

    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_v4(x_advs)
    y_incv4_after = torch.argmax(pred, dim=1)
    ta[1] += torch.count_nonzero(y_incv4_after-y_incv4)

    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_advs)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    ta[2] += torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)

## Table 2

In [ ]:
def SI_NI_TIM(x, y):
    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    eps = epsilon/255
    alpha = eps/T
    g = torch.zeros_like(x).to(device)
    x_adv = torch.clone(x)
    crit = nn.CrossEntropyLoss()
    for i in range(T):
        grads = torch.zeros_like(x).to(device)
        x_nes = x_adv + alpha*mu*g
        for j in range(m):
            x_s = (x_nes / (j+1)).requires_grad_(True)
            x_s.grad = None
            pred_1 = inc_v3(x_s)
            pred_2 = inc_v4(x_s)
            pred_3 = inc_Res_v2(x_s)
            pred_4 = resnet101(x_s)
            loss = crit(pred_1, y) + crit(pred_2, y) + crit(pred_3, y) + crit(pred_4, y)
            loss.backward()
            grads += x_s.grad
        grads /= m
        gr = torch.zeros_like(grads).to(device)
        grads = F.pad(grads, (3, 4, 3, 4), 'constant', 0)
        for j in range(7):
            for k in range(7):
                gr += grads[:, :, j:-7+j, k:-7+k]*kernel[j, k]
        grads = gr/torch.mean(torch.abs(gr), (1, 2, 3), keepdim=True)
        g = mu*g + grads
        x_adv += alpha * torch.sign(g)
        #x_adv = torch.clamp(x_adv, 0, 1)
        dif = x - x_adv
        dif = torch.clamp(dif, -eps, eps)
        x_adv = x - dif
    pred = inc_v3(x_adv)
    y_incv3_after = torch.argmax(pred, dim=1)
    pred = inc_v4(x_adv)
    y_incv4_after = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_adv)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    pred = resnet101(x_adv)
    y_resnet101_after = torch.argmax(pred, dim=1)
    return x_adv, (torch.count_nonzero(y_incv3_after-y_incv3), torch.count_nonzero(y_incv4_after-y_incv4), torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2), torch.count_nonzero(y_resnet101_after-y_resnet101))

In [ ]:
# SI_NI_TIM
ta = torch.zeros(5)
for x, y in train_loader:
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_TIM(x, y)
    ta[:-1] += torch.tensor(acc)
    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[-1] += torch.count_nonzero(y_incv3_after-y_incv3).cpu()
print(ta/1000)

In [ ]:
def SI_NI_DIM(x, y):
    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    eps = epsilon/255
    alpha = eps/T
    g = torch.zeros_like(x).to(device)
    x_adv = torch.clone(x)
    crit = nn.CrossEntropyLoss()
    for i in range(T):
        grads = torch.zeros_like(x).to(device)
        x_nes = x_adv + alpha*mu*g
        for j in range(m):
            x_s = (x_nes / (j+1))
            if torch.rand(1) > trans_prob:
                x_s = trans(x_s)
            x_s = x_s.requires_grad_(True)
            x_s.grad = None
            pred_1 = inc_v3(x_s)
            pred_2 = inc_v4(x_s)
            pred_3 = inc_Res_v2(x_s)
            pred_4 = resnet101(x_s)
            loss = crit(pred_1, y) + crit(pred_2, y) + crit(pred_3, y) + crit(pred_4, y)
            loss.backward()
            grads += x_s.grad
        grads /= m
        grads = grads/torch.mean(torch.abs(grads), (1, 2, 3), keepdim=True)
        g = mu*g + grads
        x_adv += alpha * torch.sign(g)
        #x_adv = torch.clamp(x_adv, 0, 1)
        dif = x - x_adv
        dif = torch.clamp(dif, -eps, eps)
        x_adv = x - dif
    pred = inc_v3(x_adv)
    y_incv3_after = torch.argmax(pred, dim=1)
    pred = inc_v4(x_adv)
    y_incv4_after = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_adv)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    pred = resnet101(x_adv)
    y_resnet101_after = torch.argmax(pred, dim=1)
    return x_adv, (torch.count_nonzero(y_incv3_after-y_incv3), torch.count_nonzero(y_incv4_after-y_incv4), torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2), torch.count_nonzero(y_resnet101_after-y_resnet101))

In [ ]:
# SI_NI_DIM
ta = torch.zeros(5)
for x, y in train_loader:
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_DIM(x, y)
    ta[:-1] += torch.tensor(acc)
    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[-1] += torch.count_nonzero(y_incv3_after-y_incv3).cpu()
print(ta/1000)

In [ ]:
def SI_NI_TI_DIM(x, y):
    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    eps = epsilon/255
    alpha = eps/T
    g = torch.zeros_like(x).to(device)
    x_adv = torch.clone(x)
    crit = nn.CrossEntropyLoss()
    for i in range(T):
        grads = torch.zeros_like(x).to(device)
        x_nes = x_adv + alpha*mu*g
        for j in range(m):
            x_s = (x_nes / (j+1))
            if torch.rand(1) > trans_prob:
                x_s = trans(x_s)
            x_s = x_s.requires_grad_(True)
            x_s.grad = None
            pred_1 = inc_v3(x_s)
            pred_2 = inc_v4(x_s)
            pred_3 = inc_Res_v2(x_s)
            pred_4 = resnet101(x_s)
            loss = crit(pred_1, y) + crit(pred_2, y) + crit(pred_3, y) + crit(pred_4, y)
            loss.backward()
            grads += x_s.grad
        grads /= m
        gr = torch.zeros_like(grads).to(device)
        grads = F.pad(grads, (3, 4, 3, 4), 'constant', 0)
        for j in range(7):
            for k in range(7):
                gr += grads[:, :, j:-7+j, k:-7+k]*kernel[j, k]
        grads = gr/torch.mean(torch.abs(gr), (1, 2, 3), keepdim=True)
        g = mu*g + grads
        x_adv += alpha * torch.sign(g)
        #x_adv = torch.clamp(x_adv, 0, 1)
        dif = x - x_adv
        dif = torch.clamp(dif, -eps, eps)
        x_adv = x - dif
    pred = inc_v3(x_adv)
    y_incv3_after = torch.argmax(pred, dim=1)
    pred = inc_v4(x_adv)
    y_incv4_after = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_adv)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    pred = resnet101(x_adv)
    y_resnet101_after = torch.argmax(pred, dim=1)
    return x_adv, (torch.count_nonzero(y_incv3_after-y_incv3), torch.count_nonzero(y_incv4_after-y_incv4), torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2), torch.count_nonzero(y_resnet101_after-y_resnet101))

In [ ]:
# SI_NI_TI_DIM
ta = torch.zeros(5)
for x, y in train_loader:
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_TI_DIM(x, y)
    ta[:-1] += torch.tensor(acc)
    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[-1] += torch.count_nonzero(y_incv3_after-y_incv3).cpu()
print(ta/1000)

## Figure 2 + Table 4 (NI-FGMS)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
tmi = torch.zeros((3, 7)).to(device)
tni = torch.zeros((3, 7)).to(device)
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    x = x.to(device)
    y = y.to(device)
    pred = inc_v3(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x)
    y_incv3_adv = torch.argmax(pred, dim=1)
    eps = epsilon/255
    for T in range(4, 17, 2):
        alpha = eps/T
        # MI-FGSM
        g = torch.zeros_like(x).to(device)
        x_adv = torch.clone(x)
        crit = nn.CrossEntropyLoss()
        for i in range(T):
            x_nes = torch.clone(x_adv).requires_grad_(True)
            x_nes.grad = None
            pred = inc_v3(x_nes)
            loss = crit(pred, y)
            loss.backward()
            grads = x_nes.grad
            grads = grads/torch.mean(torch.abs(grads), (1, 2, 3), keepdim=True)
            g = mu*g + grads
            x_adv += alpha * torch.sign(g)
            #x_adv = torch.clamp(x_adv, 0, 1)
            dif = x - x_adv
            dif = torch.clamp(dif, -eps, eps)
            x_adv = x - dif
        pred = inc_v3(x_adv)
        y_after = torch.argmax(pred, dim=1)
        tmi[0, (T-4)//2] += torch.count_nonzero(y_incv3-y_after)
        pred = inc_v4(x_adv)
        y_after = torch.argmax(pred, dim=1)
        tmi[1, (T-4)//2] += torch.count_nonzero(y_incv4-y_after)
        pred = inc_Res_v2(x_adv)
        y_after = torch.argmax(pred, dim=1)
        tmi[2, (T-4)//2] += torch.count_nonzero(y_inc_Res_v2-y_after)
        
        # NI-FGSM
        g = torch.zeros_like(x).to(device)
        x_adv = torch.clone(x)
        crit = nn.CrossEntropyLoss()
        for i in range(T):
            x_nes = (x_adv + alpha*mu*g).requires_grad_(True)
            x_nes.grad = None
            pred = inc_v3(x_nes)
            loss = crit(pred, y)
            loss.backward()
            grads = x_nes.grad
            grads = grads/torch.mean(torch.abs(grads), (1, 2, 3), keepdim=True)
            g = mu*g + grads
            x_adv += alpha * torch.sign(g)
            #x_adv = torch.clamp(x_adv, 0, 1)
            dif = x - x_adv
            dif = torch.clamp(dif, -eps, eps)
            x_adv = x - dif
        pred = inc_v3(x_adv)
        y_incv3_after = torch.argmax(pred, dim=1)
        tni[0, (T-4)//2] += torch.count_nonzero(y_incv3-y_incv3_after)
        pred = inc_v4(x_adv)
        y_incv4_after = torch.argmax(pred, dim=1)
        tni[1, (T-4)//2] += torch.count_nonzero(y_incv4-y_incv4_after)
        pred = inc_Res_v2(x_adv)
        y_inc_Res_v2_after = torch.argmax(pred, dim=1)
        tni[2, (T-4)//2] += torch.count_nonzero(y_inc_Res_v2-y_inc_Res_v2_after)
        if T == 10:
            pred = resnet101(x_adv)
            y_resnet101_after = torch.argmax(pred, dim=1)
            pred = inc_v3_adv(x_adv)
            y_incv3_adv_after = torch.argmax(pred, dim=1)
            ta[0] += torch.count_nonzero(y_incv3_after-y_incv3)
            ta[1] += torch.count_nonzero(y_incv4_after-y_incv4)
            ta[2] += torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2)
            ta[3] += torch.count_nonzero(y_resnet101_after-y_resnet101)
            ta[4] += torch.count_nonzero(y_incv3_adv_after-y_incv3_adv)
for i in range(3):
    plt.plot(range(4, 17, 2), tmi[i].cpu()/1000, label='MI-FGSM')
    plt.plot(range(4, 17, 2), tni[i].cpu()/1000, label='NI-FGSM')
    plt.xlabel('Iteration Number')
    plt.xlim(4, 16)
    plt.ylabel('Attack Success Rate')
    plt.legend()
    plt.show()
print(ta.cpu()/1000)
T = 10

## Table 4 (SI-NI-FGSM)

In [31]:
def SI_NI_FGSM(x, y, model):
    pred = model(x)
    y_pred = torch.argmax(pred, dim=1)
    eps = epsilon/255
    alpha = eps/T
    g = torch.zeros_like(x).to(device)
    x_adv = torch.clone(x)
    crit = nn.CrossEntropyLoss()
    for i in range(T):
        grads = torch.zeros_like(x).to(device)
        x_nes = x_adv + alpha*mu*g
        for j in range(m):
            x_s = (x_nes / (j+1)).requires_grad_(True)
            x_s.grad = None
            pred = model(x_s)
            loss = crit(pred, y)
            loss.backward()
            grads += x_s.grad
        grads /= m
        grads = grads/torch.mean(torch.abs(grads), (1, 2, 3), keepdim=True)
        g = mu*g + grads
        x_adv += alpha * torch.sign(g)
        dif = x - x_adv
        dif = torch.clamp(dif, -eps, eps)
        x_adv = x - dif
    pred = model(x_adv)
    y_guess = torch.argmax(pred, dim=1)
    return x_adv, torch.count_nonzero(y_guess-y_pred)

In [ ]:
ta = torch.zeros(5).to(device)
for x, y in train_loader:
    
    torch.cuda.empty_cache()
    x = x.to(device)
    y = y.to(device)
    x_advs, acc = SI_NI_FGSM(x.to(device), y.to(device), inc_v3)
    ta[0] += acc

    pred = inc_v4(x)
    y_incv4 = torch.argmax(pred, dim=1)
    pred = inc_v4(x_advs)
    y_incv4_after = torch.argmax(pred, dim=1)
    ta[1] += torch.count_nonzero(y_incv4_after-y_incv4)

    pred = inc_Res_v2(x)
    y_inc_Res_v2 = torch.argmax(pred, dim=1)
    pred = inc_Res_v2(x_advs)
    y_inc_Res_v2_after = torch.argmax(pred, dim=1)
    ta[2] += torch.count_nonzero(y_inc_Res_v2_after-y_inc_Res_v2)

    pred = resnet101(x)
    y_resnet101 = torch.argmax(pred, dim=1)
    pred = resnet101(x_advs)
    y_resnet101_after = torch.argmax(pred, dim=1)
    ta[3] += torch.count_nonzero(y_resnet101_after-y_resnet101)

    pred = inc_v3_adv(x)
    y_incv3 = torch.argmax(pred, dim=1)
    pred = inc_v3_adv(x_advs)
    y_incv3_after = torch.argmax(pred, dim=1)
    ta[4] += torch.count_nonzero(y_incv3_after-y_incv3)
print(ta.cpu()/1000)